In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import sep

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

In [ ]:
data = image_data.byteswap(False).newbyteorder()

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation='nearest', cmap='inferno', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

In [ ]:
bkg = sep.Background(data)
#Input array with dtype '>f4' has non-native byte order. Only native byte order arrays are supported
#To change the byte order of the array 'data', do 'data = data.byteswap().newbyteorder()'

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)
#name 'globalback' is not defined

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='inferno', origin='lower')
plt.colorbar();

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='inferno', origin='lower')
plt.colorbar();

In [ ]:
data_sub = data - bkg

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='inferno', vmin=m-s, vmax=m+s,
               origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
               width=6*objects['a'][i], height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0,
                                    err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
n_bins = 8640
x = flux, fluxerr, flag

In [ ]:
fig = plt.figure(figsize=(12,10))
y_hist, x_hist, ignored = plt.hist(x, bins=n_bins, range=[-5,5], density=True)
xx = np.linspace(-5.0,5.0,1000)
plt.ylim([0,15])
plt.xlim([-0.10,2.25])
plt.gca().set_aspect(.25)
plt.xlabel('x')
plt.ylabel('y(x)')

plt.show()